In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

# On vérifie que les fichiers train.csv et test.csv sont bien chargés.
# print(df_train) 
# print(df_test)

# Afficher les types de données des colonnes
print('Afficher les types de données des colonnes :')
print(df_train.dtypes)
print(df_test.dtypes)

print('---------------------------------------')

# Vérifier les valeurs manquantes dans le DataFrame et les additionner
print('Vérifier les valeurs manquantes additionnées dans le DataFrame :')
print(df_train.isna().sum())
print(df_test.isna().sum())

print('---------------------------------------')

In [ ]:
# Afficher les statistiques descriptives du DataFrame
print('Statistiques descriptives du DataFrame :')
print(df_train.describe())
print(df_test.describe())

In [ ]:
# Afficher les informations du DataFrame
print('Informations sur le DataFrame :')
df_train.info()
df_test.info()

print('---------------------------------------')

# Afficher les 5 premières lignes du DataFrame
print('Les 5 premières lignes du DataFrame :')
df_train.head(5)
df_test.head(5)

In [ ]:
# Filtrer les passagers qui n'ont pas survécu
df_train_filtered = df_train.query('Survived == False')

print(df_train_filtered)

In [ ]:
# Répartition des passagers par tranches d'âges
fig_reparti_age_passager = px.histogram(df_train, x='Age', nbins=30, title='Répartition des passagers par tranches d\'âge')
fig_reparti_age_passager.update_layout(xaxis_title='Âge', yaxis_title='Nombre de passagers')
fig_reparti_age_passager.show() 

In [ ]:
# Compter le nombre d'hommes et de femmes sur le Titanic

# Créer une nouvelle colonne avec des labels explicites
df_train['Sex_label'] = df_train['Sex'].map({'male': 'Homme', 'female': 'Femme'})
df_test['Sex_label'] = df_test['Sex'].map({'male': 'Homme', 'female': 'Femme'})

# Afficher la répartition des passagers par sexe
fig_total_sexe = px.histogram(
    df_train,
    x='Sex_label',
    color='Sex_label',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    labels={'Sex_label': 'Sexe'},
    title="Répartition des passagers par sexe",
)

# Ajouter le nombre de passagers par sexe
fig_total_sexe.update_layout(
    xaxis_title="Sexe",
    yaxis_title="Nombre de passagers",
    bargap=0.4,
    height=600
)

# Afficher le graphique
fig_total_sexe.show()

In [ ]:
# Compter le nombre de survivants et de décès selon le sexe

# Créer une nouvelle colonne avec des labels explicites
df_train['Survived_label'] = df_train['Survived'].map({0: 'Mort', 1: 'Survécu'})
df_test['Survived_label'] = df_test['Survived'].map({0: 'Mort', 1: 'Survécu'})

# Créer le graphique de la répartition des survivants et des décès selon le sexe
fig_survie_sexe = px.histogram( 
    df_train,
    x='Sex_label',
    color='Survived_label',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    title="Nombre de survivants et de décès selon le sexe",
    labels={'Survived_label': 'Statut de survie'},
    category_orders={
        'Sex_label': ['Homme', 'Femme'],
        'Survived_label': ['Survécu', 'Mort']
    }
)

# Update le graphique avec les titres des axes et la hauteur
fig_survie_sexe.update_layout(
    xaxis_title="Sexe",
    yaxis_title="Nombre de passagers",
    bargap=0.4,
    height=600
)

# Afficher le graphique
fig_survie_sexe.show()

In [ ]:
# 1. Créer des tranches d'âge larges (10 ans)
bins = range(0, 90, 10)
labels = [f'{i}-{i+10}' for i in bins[:-1]]
df_train['AgeBin'] = pd.cut(df_train['Age'], bins=bins, labels=labels)
df_test['AgeBin'] = pd.cut(df_train['Age'], bins=bins, labels=labels)

# 2. Grouper par AgeBin et Sex
grouped = df_train.groupby(['AgeBin', 'Sex', 'Survived']).size().reset_index(name='Count')
grouped_test = df_test.groupby(['AgeBin', 'Sex', 'Survived']).size().reset_index(name='Count')

# 3. Total par AgeBin + Sex (pourcentage global sur chaque groupe)
grouped['Total'] = grouped.groupby(['AgeBin', 'Sex'])['Count'].transform('sum')
grouped['Percentage'] = grouped['Count'] / grouped['Total'] * 100

grouped_test['Total'] = grouped_test.groupby(['AgeBin', 'Sex'])['Count'].transform('sum')
grouped_test['Percentage'] = grouped_test['Count'] / grouped_test['Total'] * 100

# 4. Remplacement de Survived pour affichage
grouped['Survived'] = grouped['Survived'].map({0: 'Mort', 1: 'Vivant'})

# 5. Ordre personnalisé
category_order = {'AgeBin': labels, 'Survived': ['Mort', 'Vivant']}

# 6. Affichage en barres empilées (stacked)
fig_survived_per_age = px.bar(
    grouped,
    x='AgeBin',
    y='Percentage',
    color='Survived',
    barmode='stack',
    facet_col='Sex',
    labels={
        'AgeBin': 'Tranche d\'âge',
        'Percentage': 'Pourcentage (stacked)',
        'Survived': 'Statut'
    },
    title='Répartition des morts et survivants par tranche d’âge et par sexe',
    category_orders=category_order,
    color_discrete_map={
        'Mort': '#d62728',    # rouge
        'Vivant': '#1f77b4'   # bleu
    }
)

fig_survived_per_age.update_layout(
    yaxis_title='Pourcentage (100% par tranche d’âge et sexe)',
    xaxis_title='Tranche d\'âge',
    bargap=0.05
)

fig_survived_per_age.show()

In [ ]:
# Répartition de la survie selon le sexe (graphique circulaire et pourcentages)

# Données hommes et femmes
df_hommes = df_test[df_test['Sex'] == 'male']
df_femmes = df_test[df_test['Sex'] == 'female']

df_hommes_test = df_test[df_test['Sex'] == 'male']
df_femmes_test = df_test[df_test['Sex'] == 'female']

# Compter les occurrences
hommes_counts = df_hommes['Survived_label'].value_counts().reindex(['Survécu', 'Mort'], fill_value=0)
femmes_counts = df_femmes['Survived_label'].value_counts().reindex(['Survécu', 'Mort'], fill_value=0)

hommes_counts = df_hommes_test['Survived_label'].value_counts().reindex(['Survécu', 'Mort'], fill_value=0)
femmes_counts = df_femmes_test['Survived_label'].value_counts().reindex(['Survécu', 'Mort'], fill_value=0)

# Créer le graphique circulaire pour la répartition de la survie selon le sexe
fig_camembert = make_subplots(        # Création de la figure avec deux domaines pour les camenberts
    rows=1, cols=2,
    specs=[[{'type': 'domain'}, {'type': 'domain'}]],
    subplot_titles=["Hommes", "Femmes"]
)

# Définir les couleurs dans l’ordre des labels : ['Survécu', 'Mort']
colors_inverses = ['blue', 'red']  # Survécu = rouge, Mort = bleu

# Ajout du graphique pour les hommes
fig_camembert.add_trace(go.Pie(
    labels=hommes_counts.index,
    values=hommes_counts.values,
    name="Hommes",
    marker=dict(colors=colors_inverses),
    textinfo='percent+label'
), row=1, col=1)

# Ajout du graphique pour les femmes
fig_camembert.add_trace(go.Pie(
    labels=femmes_counts.index,
    values=femmes_counts.values,
    name="Femmes",
    marker=dict(colors=colors_inverses),
    textinfo='percent+label'
), row=1, col=2)

# Mise en page finale
fig_camembert.update_layout(
    title_text="Répartition de la survie selon le sexe",
    title_x=0.5,
    height=500
)

fig_camembert.show()

In [ ]:
# Répartition en pourcentage des passagers par classe et survie
# Préparation des données groupées
grouped = (
    df_train
    .groupby(['Pclass', 'Survived'])
    .size()
    .reset_index(name='Count')
)

grouped_test = (
    df_test
    .groupby(['Pclass', 'Survived'])
    .size()
    .reset_index(name='Count')
)

# Calcul du pourcentage par classe
grouped['Total'] = grouped.groupby('Pclass')['Count'].transform('sum')
grouped['Pourcentage'] = 100 * grouped['Count'] / grouped['Total']

grouped_test['Total'] = grouped_test.groupby('Pclass')['Count'].transform('sum')
grouped_test['Pourcentage'] = 100 * grouped_test['Count'] / grouped['Total']

# Création des labels
grouped['Survived_label'] = grouped['Survived'].map({0: 'Mort', 1: 'Vivant'})
grouped['Pclass'] = grouped['Pclass'].astype(str)

grouped_test['Survived_label'] = grouped_test['Survived'].map({0: 'Mort', 1: 'Vivant'})
grouped_test['Pclass'] = grouped_test['Pclass'].astype(str)

# Création du graphique
fig_survie_par_classe = px.bar(
    grouped,
    x='Pclass',
    y='Pourcentage',
    color='Survived_label',
    text=grouped['Pourcentage'].round(1).astype(str) + '%',
    color_discrete_map={'Mort': 'lightcoral','Vivant': 'lightgreen'},
    barmode='stack',
    labels={
        'Pclass': 'Classe',
        'Pourcentage': 'Pourcentage (%)',
        'Survived_label': 'Statut de survie'
    },
    title='Répartition en % des passagers par classe et survie'
)

fig_survie_par_classe.update_layout(
    yaxis_title='Pourcentage (%)',
    xaxis_title='Classe',
    height=600,
    bargap=0.5
)

fig_survie_par_classe.show()

In [ ]:
# Répartition des passagers par classe et par sexe
sns.countplot(data=df_train, x="Pclass", hue="Sex")
plt.title("Répartition des passagers par classe et sexe")
plt.xlabel("Classe")
plt.ylabel("Nombre de passagers")
plt.show()

In [ ]:
# Voir la relation entre la classe et le lieu d'embarcation.
sns.countplot(data=df_train,x="Embarked",hue="Pclass", dodge=True,palette="Set2")
plt.title("Répartition des classes par port d'embarquement")
plt.xlabel("Port d'embarquement")
plt.ylabel("Nombre de passagers")
plt.show()

In [ ]:
# Création des colonnes utiles
df_train['Accompanied'] = ((df_train['SibSp'] + df_train['Parch']) > 0).astype(int)
df_train['Survived_label'] = df_train['Survived'].map({0: 'Mort', 1: 'Vivant'})
df_train['Accompanied_label'] = df_train['Accompanied'].map({0: 'Non', 1: 'Oui'})
df_train['Pclass'] = df_train['Pclass'].astype(int)
df_train['Sex_label'] = df_train['Sex'].map({'male': 'Hommes', 'female': 'Femmes'})

df_test['Accompanied'] = ((df_test['SibSp'] + df_test['Parch']) > 0).astype(int)
df_test['Survived_label'] = df_test['Survived'].map({0: 'Mort', 1: 'Vivant'})
df_test['Accompanied_label'] = df_test['Accompanied'].map({0: 'Non', 1: 'Oui'})
df_test['Pclass'] = df_test['Pclass'].astype(int)
df_test['Sex_label'] = df_test['Sex'].map({'male': 'Hommes', 'female': 'Femmes'})

# Séparation hommes / femmes
df_hommes = df_train[df_train['Sex_label'] == 'Hommes']
df_femmes = df_train[df_train['Sex_label'] == 'Femmes']

df_hommes = df_test[df_test['Sex_label'] == 'Hommes']
df_femmes = df_test[df_test['Sex_label'] == 'Femmes']

# Cette fonction prend un DataFrame et retourne un DataFrame avec les pourcentages de survie selon l'accompagnement et la classe
# Elle regroupe les données par classe, accompagnement et survie, puis calcule le pourcentage de survie pour chaque groupe
def create_percentage_df(df):
    grouped = (
        df
        .groupby(['Pclass', 'Accompanied_label', 'Survived_label'])
        .size()
        .reset_index(name='Count')
    )
    grouped['Total'] = grouped.groupby(['Pclass', 'Accompanied_label'])['Count'].transform('sum')
    grouped['Pourcentage'] = 100 * grouped['Count'] / grouped['Total']
    return grouped

# Calcul des pourcentages
grouped_hommes = create_percentage_df(df_hommes)
grouped_femmes = create_percentage_df(df_femmes)

# Graphique hommes
fig_hommes = px.bar(
    grouped_hommes,
    x='Accompanied_label',
    y='Pourcentage',
    color='Survived_label',
    color_discrete_map={'Mort': 'lightcoral','Vivant': 'lightgreen'},
    facet_col='Pclass',
    barmode='stack',
    text=grouped_hommes['Pourcentage'].round(1).astype(str) + '%',
    labels={
        'Accompanied_label': 'Accompagné',
        'Pourcentage': 'Pourcentage',
        'Survived_label': 'Statut de survie',
        'Pclass': 'Classe'
    },
    title='Hommes : taux de survie selon l\'accompagnement et la classe (en %)'
)

# Graphique femmes
fig_femmes = px.bar(
    grouped_femmes,
    x='Accompanied_label',
    y='Pourcentage',
    color='Survived_label',
    color_discrete_map={'Mort': 'lightcoral','Vivant': 'lightgreen'},
    facet_col='Pclass',
    barmode='stack',
    text=grouped_femmes['Pourcentage'].round(1).astype(str) + '%',
    labels={
        'Accompanied_label': 'Accompagné',
        'Pourcentage': 'Pourcentage',
        'Survived_label': 'Statut de survie',
        'Pclass': 'Classe'
    },
    title='Femmes : taux de survie selon l\'accompagnement et la classe (en %)'
)

# Mise en page des graphiques
for fig in [fig_hommes, fig_femmes]:
    fig.update_layout(
        yaxis_title='Pourcentage (%)',
        xaxis_title='Accompagné',
        height=600,
        bargap=0.3
    )

fig_hommes.show()
fig_femmes.show()

In [ ]:
# S'assurer que la colonne Accompanied est bien définie
df_train['Accompanied'] = ((df_train['SibSp'] + df_train['Parch']) > 0).astype(int)

df_test['Accompanied'] = ((df_test['SibSp'] + df_test['Parch']) > 0).astype(int)

# Calcul de la médiane de l’âge par combinaison Sex / Pclass / Accompanied
median_age = df_train.groupby(['Sex', 'Pclass', 'Accompanied'])['Age'].median()
median_age_test = df_test.groupby(['Sex', 'Pclass', 'Accompanied'])['Age'].median()

# Affichage du résultat
print(median_age)
print(median_age_test)

In [ ]:
# Transformer la colonne 'Sex' en entier binaire : male = 1, female = 0
df_train['male'] = (df_train['Sex'] == 'male').astype(int)
df_test['male'] = (df_test['Sex'] == 'male').astype(int)

# Supprimer l’ancienne colonne 'Sex'
# df_train = df_train.drop(columns=['Sex'])  # optionnel

# Vérifier le résultat
print(df_train[['male']].head())

In [ ]:
# Vérification des valeurs manquantes dans la colonne 'Embarked'

# Affiche les lignes concernées
df_train[df_train['Embarked'].isna()]
df_test[df_test['Embarked'].isna()]

# On sait que la classe était 1 et le billet coûte 80
# On va borner avec les valeurs connus pour trouver des concordances 
df_train[
    (df_train['Pclass'] == 1) &     # On veut cibler les personnes de la classe 1
    (df_train['Fare'].between(75, 85)) &    # On borne dans un tarif compris entre 75 et 85 
    (df_train['Embarked'].notna()) &    # On veut prendre que les valeurs connues dans 'Embarked' 
    (df_train['Sex'] == 'female')     # On filtre que pour les femmes
][['Embarked', 'Fare', 'Pclass', 'Sex']]['Embarked'].value_counts()  # On prend en compte toutes les sélections et on compte le nombre d'occurence identiques pour 'Embarked'

df_test[
    (df_test['Pclass'] == 1) &     # On veut cibler les personnes de la classe 1
    (df_test['Fare'].between(75, 85)) &    # On borne dans un tarif compris entre 75 et 85 
    (df_test['Embarked'].notna()) &    # On veut prendre que les valeurs connues dans 'Embarked' 
    (df_test['Sex'] == 'female')     # On filtre que pour les femmes
][['Embarked', 'Fare', 'Pclass', 'Sex']]['Embarked'].value_counts()  # On prend en compte toutes les sélections et on compte le nombre d'occurence identiques pour 'Embarked'

# On remplace la valeur NaN par la valeur 'C' suite à l'analyse précédente
df_train['Embarked'] = df_train['Embarked'].fillna('C')
df_test['Embarked'] = df_train['Embarked'].fillna('C')

# Compléter les valeurs manquantes de 'Embarked' par analyse contextuelle
df_train.loc[df_train['Embarked'].isna(), 'Embarked'] = 'C'
df_test.loc[df_test['Embarked'].isna(), 'Embarked'] = 'C'

In [ ]:
# On vérifie les titres de chaque passager

# Extraire le titre avec une regex
df_train['Title'] = df_train['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
df_test['Title'] = df_test['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)

# On regroupe les titres uniques entre-eux
df_train['Title'].value_counts()
df_test['Title'].value_counts()

In [ ]:
# Calcul de la médiane de l’âge par combinaison Sex / Pclass / Accompanied
df_median=df_train.copy()

median_age_simple = (
    df_median
    .groupby(['Sex', 'Pclass', 'Accompanied'])['Age']
    .median()
    .reset_index()
    .sort_values(by=['Sex', 'Pclass', 'Accompanied'])
    .rename(columns={
        'Sex': 'Sexe',
        'Pclass': 'Classe',
        'Accompanied': 'Accompagné',
        'Age': 'Âge médian'
    })
)

# Affichage propre du tableau
print("\nMédiane de l'âge selon le sexe, la classe et l'accompagnement :\n")
print(median_age_simple.to_string(index=False))

In [ ]:
# Médiane de l'âge par combinaison Sex / Pclass / Accompanied
median_age_simple = (
    df_median
    .groupby(['Sex', 'Pclass', 'Accompanied'])['Age']
    .median()
    .reset_index()
    .rename(columns={
        'Sex': 'Sexe',
        'Pclass': 'Classe',
        'Accompanied': 'Accompagné',
        'Age': 'Âge médian'
    })
)

print("\nMédiane de l'âge selon le sexe, la classe et l'accompagnement :\n")
print(median_age_simple.sort_values(by=['Sexe', 'Classe', 'Accompagné']).to_string(index=False))

# Médiane détaillée d’âge selon Sex + Title + Pclass + Accompanied (pour les titres les plus fréquents)
selected_titles = ['Master', 'Mrs', 'Miss', 'Mr']
df_median_filtered = df_median[df_median['Title'].isin(selected_titles)]

median_age_detailed = (
    df_median_filtered
    .groupby(['Sex', 'Title', 'Pclass', 'Accompanied'])['Age']
    .median()
    .reset_index()
    .rename(columns={'Age': 'Age_median'})
    .sort_values(by=['Sex', 'Title', 'Pclass', 'Accompanied'])
)

print("\nMédiane de l'âge selon le sexe, le titre, la classe et l'accompagnement :\n")
print(median_age_detailed.to_string(index=False))

# Création du dictionnaires pour l'imputation
# (on garde le dictionnaire par Titre/Classe/Sexe mais on ne l'affiche plus)
median_age_title = (
    df_median_filtered
    .groupby(['Title', 'Pclass', 'Sex'])['Age']
    .median()
    .reset_index()
    .rename(columns={
        'Title': 'Titre',
        'Pclass': 'Classe',
        'Sex': 'Sexe',
        'Age': 'Âge médian'
    })
)

median_age_title_dict = median_age_title.set_index(
    ['Titre', 'Classe', 'Sexe']
)['Âge médian'].to_dict()

median_age_detailed_dict = median_age_detailed.set_index(
    ['Sex', 'Title', 'Pclass', 'Accompanied']
)['Age_median'].to_dict()

median_age_simple_dict = median_age_simple.set_index(
    ['Sexe', 'Classe', 'Accompagné']
)['Âge médian'].to_dict()

# Fonction d'imputation d'âge
def impute_age(row):
    if pd.notna(row['Age']):
        return row['Age']

    # Médiane détaillée (Sex, Title, Pclass, Accompanied)
    key_detailed = (row['Sex'], row['Title'], row['Pclass'], row['Accompanied'])
    if key_detailed in median_age_detailed_dict:
        return median_age_detailed_dict[key_detailed]

    # Médiane par Titre, Classe, Sexe
    key_title = (row['Title'], row['Pclass'], row['Sex'])
    if key_title in median_age_title_dict:
        return median_age_title_dict[key_title]

    # Médiane simple par Sex, Classe, Accompagné
    key_simple = (row['Sex'], row['Pclass'], row['Accompanied'])
    if key_simple in median_age_simple_dict:
        return median_age_simple_dict[key_simple]

    # Médiane générale
    return df_train['Age'].median()

# Application de l’imputation
df_median['Age'] = df_median.apply(impute_age, axis=1)

print("\nExemple passager index 766 :")
print(df_median.loc[766, ['Name', 'Sex', 'Pclass', 'Accompanied', 'Title', 'Age']])

# Répartition des passagers par tranches d'âges
fig_reparti_age_passager = px.histogram(df_train, x='Age', nbins=30, title='Répartition des passagers par tranches d\'âge')
fig_reparti_age_passager.update_layout(xaxis_title='Âge', yaxis_title='Nombre de passagers')
fig_reparti_age_passager.show() 

# Répartition des passagers par tranches d'âges après imputation
fig_reparti_age_passager_fill = px.histogram(df_median, x='Age', nbins=30, title='Répartition des passagers par tranches d\'âge après imputation')
fig_reparti_age_passager_fill.update_layout(xaxis_title='Âge', yaxis_title='Nombre de passagers')
fig_reparti_age_passager_fill.show() 


In [ ]:
# Suppression des colonnes non utiles

df_median = df_median.drop(['Sex_label', 'Survived_label', 'AgeBin', 'Cabin', 'Ticket', 'Sex', 'Name', 'Accompanied_label'], axis=1)
print(df_median.info())

In [ ]:
# Encodage des colonnes categoriques en variables numériques

df_median = pd.get_dummies(df_median, columns=['Title', 'Embarked'], drop_first=False)
print(df_median.info())

In [ ]:
# Appliquer les transformations avec le df_median à df_train

df_train = df_median
print(df_train.info())

In [ ]:
# Visualisation de la log loss et de la précision (accuracy) durant l'entraînement

import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
from sklearn.utils import shuffle
import numpy as np

# Étape 1 : Séparer les features (X) et la cible (y)
X = df_train.drop(columns=['Survived'])  # Toutes les colonnes sauf la cible
y = df_train['Survived']  # La colonne à prédire (0 = mort, 1 = survécu)

# Étape 2 : Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Étape 3 : Mélanger les données pour un apprentissage plus robuste
X_shuffled, y_shuffled = shuffle(X_scaled, y, random_state=42)

# Étape 4 : Initialiser le modèle SGD
clf = SGDClassifier(
    loss='log_loss',
    learning_rate='constant',
    alpha=0.01,
    eta0=0.01,
    max_iter=1,
    warm_start=True,
    random_state=42,
    penalty='elasticnet'
)

# Étape 5 : Boucle d'entraînement
losses = []      # Liste des log loss
accuracies = []  # Liste des précisions
n_epochs = 20    # Nombre d'époques

for epoch in range(n_epochs):
    # Entraînement sur tout le jeu de données (en mode incrémental)
    clf.partial_fit(X_shuffled, y_shuffled, classes=np.unique(y))

    # Prédiction des probabilités (pour log loss)
    y_proba = clf.predict_proba(X_shuffled)
    epoch_loss = log_loss(y_shuffled, y_proba)
    losses.append(epoch_loss)

    # Prédiction des classes (pour accuracy)
    y_pred = clf.predict(X_shuffled)
    epoch_acc = accuracy_score(y_shuffled, y_pred)
    accuracies.append(epoch_acc)

# Étape 6 : Affichage des deux courbes sur le même graphique
plt.figure(figsize=(12, 6))

# Courbe de log loss
plt.plot(range(1, n_epochs + 1), losses, marker='o', color='blue', label='Log loss')

# Courbe de précision
plt.plot(range(1, n_epochs + 1), accuracies, marker='s', color='green', label='Accuracy')

plt.title("Évolution de la log loss et de la précision durant l'entraînement")
plt.xlabel("Époque")
plt.ylabel("Valeur")
plt.legend()
plt.grid(True)
plt.xticks(range(1, n_epochs + 1))
plt.show()

# Moyenne de la log loss
avg_loss = np.mean(losses)
print(f"Moyenne de la log loss sur {n_epochs} époques : {avg_loss:.4f}")

# Moyenne de l'accuracy
avg_accuracy = np.mean(accuracies)
print(f"Moyenne de la précision (accuracy) sur {n_epochs} époques : {avg_accuracy:.4f}")

# Prédictions sur les données de test
X_test =        # le jeu de données TEST sans la variable cible 
y_test =       # variable cible/target de TEST
y_pred = model.predict(X_test)

# 5. Évaluation du modèle : calcul de la précision (accuracy)
accuracy = accuracy_score(y_test, y_pred)


# 6. Affichage du résultat
print(f"Accuracy du modèle SVC linéaire : {accuracy}")
